<a href="https://colab.research.google.com/github/Rayza126/Pine-City-Zoo/blob/main/EPL_PREDICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
from google.colab import files
uploaded = files.upload()


Saving matches.csv to matches.csv


In [12]:
matches = pd.read_csv('matches.csv', index_col=0)

In [13]:
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2025-08-17,16:30,Premier League,Matchweek 1,Sun,Away,W,1.0,0.0,Manchester Utd,...,Match Report,NaN,9.0,3.0,18.6,1.0,0,0,2025,Arsenal
1,2025-08-23,17:30,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Leeds United,...,Match Report,NaN,17.0,4.0,15.2,0.0,1,1,2025,Arsenal
2,2025-08-31,16:30,Premier League,Matchweek 3,Sun,Away,L,0.0,1.0,Liverpool,...,Match Report,NaN,11.0,1.0,17.5,1.0,0,0,2025,Arsenal
3,2025-09-13,12:30,Premier League,Matchweek 4,Sat,Home,W,3.0,0.0,Nott'ham Forest,...,Match Report,NaN,16.0,5.0,13.5,0.0,0,0,2025,Arsenal
5,2025-09-21,16:30,Premier League,Matchweek 5,Sun,Home,D,1.0,1.0,Manchester City,...,Match Report,NaN,12.0,3.0,14.1,0.0,0,0,2025,Arsenal


In [14]:
matches.shape

(220, 28)

In [19]:
matches['Team'].value_counts()

,count
Team,
Arsenal,11
Manchester City,11
Chelsea,11
Sunderland,11
Tottenham Hotspur,11
Aston Villa,11
Manchester United,11
Liverpool,11
Bournemouth,11


In [20]:
matches.dtypes

,0
Date,object
Time,object
Comp,object
Round,object
Day,object
Venue,object
Result,object
GF,float64
GA,float64
Opponent,object


In [21]:
matches['Date'] = pd.to_datetime(matches['Date'])

In [22]:
#creating predictors for machine learning
matches['Venue_code'] = matches['Venue'].astype('category').cat.codes

In [23]:
matches['Opp_code'] = matches['Opponent'].astype('category').cat.codes

In [24]:
matches['hour'] = matches['Time'].str.replace(":.+", "", regex=True).astype('int')

In [25]:
matches['day_code'] = matches['Date'].dt.dayofweek

In [26]:
matches["target"] = (matches["Result"] == "W").astype("int")

In [27]:
matches

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,FK,PK,PKatt,Season,Team,Venue_code,Opp_code,hour,day_code,target
0,2025-08-17,16:30,Premier League,Matchweek 1,Sun,Away,W,1.0,0.0,Manchester Utd,...,1.0,0,0,2025,Arsenal,0,13,16,6,1
1,2025-08-23,17:30,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Leeds United,...,0.0,1,1,2025,Arsenal,1,10,17,5,1
2,2025-08-31,16:30,Premier League,Matchweek 3,Sun,Away,L,0.0,1.0,Liverpool,...,1.0,0,0,2025,Arsenal,0,11,16,6,0
3,2025-09-13,12:30,Premier League,Matchweek 4,Sat,Home,W,3.0,0.0,Nott'ham Forest,...,0.0,0,0,2025,Arsenal,1,15,12,5,1
5,2025-09-21,16:30,Premier League,Matchweek 5,Sun,Home,D,1.0,1.0,Manchester City,...,0.0,0,0,2025,Arsenal,1,12,16,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,2025-10-05,14:00,Premier League,Matchweek 7,Sun,Home,D,1.0,1.0,Brighton,...,1.0,0,0,2025,Wolverhampton Wanderers,1,4,14,6,0
9,2025-10-18,15:00,Premier League,Matchweek 8,Sat,Away,L,0.0,2.0,Sunderland,...,1.0,0,0,2025,Wolverhampton Wanderers,0,16,15,5,0
10,2025-10-26,14:00,Premier League,Matchweek 9,Sun,Home,L,2.0,3.0,Burnley,...,1.0,1,1,2025,Wolverhampton Wanderers,1,5,14,6,0
12,2025-11-01,15:00,Premier League,Matchweek 10,Sat,Away,L,0.0,3.0,Fulham,...,1.0,0,0,2025,Wolverhampton Wanderers,0,9,15,5,0


In [28]:
#Training our model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [36]:
train = matches[matches["Date"] < '2025-11-01']

In [38]:
test = matches[matches["Date"] >= '2025-11-01']

In [39]:
predictors = ['Venue_code', 'Opp_code', 'hour', 'day_code']

In [40]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [49]:
predictions = rf.predict(test[predictors])

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
accuracy = accuracy_score(test["target"], predictions)

In [52]:
accuracy

0.625

In [53]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=predictions))

In [55]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,18,7
1,8,7


In [59]:
from sklearn.metrics import precision_score

In [60]:
precision_score(test["target"], predictions)

0.5

In [61]:
# Improving our model
grouped_matches = matches.groupby("Team")

In [62]:
group = grouped_matches.get_group("Liverpool").sort_values("Date")

In [63]:
group

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,FK,PK,PKatt,Season,Team,Venue_code,Opp_code,hour,day_code,target
1,2025-08-15,20:00,Premier League,Matchweek 1,Fri,Home,W,4.0,2.0,Bournemouth,...,0.0,0,0,2025,Liverpool,1,2,20,4,1
2,2025-08-25,20:00,Premier League,Matchweek 2,Mon,Away,W,3.0,2.0,Newcastle Utd,...,0.0,0,0,2025,Liverpool,0,14,20,0,1
3,2025-08-31,16:30,Premier League,Matchweek 3,Sun,Home,W,1.0,0.0,Arsenal,...,1.0,0,0,2025,Liverpool,1,0,16,6,1
4,2025-09-14,14:00,Premier League,Matchweek 4,Sun,Away,W,1.0,0.0,Burnley,...,1.0,1,1,2025,Liverpool,0,5,14,6,1
6,2025-09-20,12:30,Premier League,Matchweek 5,Sat,Home,W,2.0,1.0,Everton,...,1.0,0,0,2025,Liverpool,1,8,12,5,1
8,2025-09-27,15:00,Premier League,Matchweek 6,Sat,Away,L,1.0,2.0,Crystal Palace,...,1.0,0,0,2025,Liverpool,0,7,15,5,0
10,2025-10-04,17:30,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Chelsea,...,0.0,0,0,2025,Liverpool,0,6,17,5,0
11,2025-10-19,16:30,Premier League,Matchweek 8,Sun,Home,L,1.0,2.0,Manchester Utd,...,0.0,0,0,2025,Liverpool,1,13,16,6,0
13,2025-10-25,20:00,Premier League,Matchweek 9,Sat,Away,L,2.0,3.0,Brentford,...,0.0,0,0,2025,Liverpool,0,3,20,5,0
15,2025-11-01,20:00,Premier League,Matchweek 10,Sat,Home,W,2.0,0.0,Aston Villa,...,1.0,0,0,2025,Liverpool,1,1,20,5,1


In [70]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [74]:
cols = ["GF", "GA", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [75]:
rolling_averages(group, cols, new_cols)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,day_code,target,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,FK_rolling,PK_rolling,PKatt_rolling
4,2025-09-14,14:00,Premier League,Matchweek 4,Sun,Away,W,1.0,0.0,Burnley,...,6,1,2.666667,1.333333,11.000000,5.666667,19.433333,0.333333,0.000000,0.000000
6,2025-09-20,12:30,Premier League,Matchweek 5,Sat,Home,W,2.0,1.0,Everton,...,5,1,1.666667,0.666667,13.333333,3.333333,20.066667,0.666667,0.333333,0.333333
8,2025-09-27,15:00,Premier League,Matchweek 6,Sat,Away,L,1.0,2.0,Crystal Palace,...,5,0,1.333333,0.333333,15.333333,3.000000,19.366667,1.000000,0.333333,0.333333
10,2025-10-04,17:30,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Chelsea,...,5,0,1.333333,1.000000,19.000000,3.333333,17.066667,1.000000,0.333333,0.333333
11,2025-10-19,16:30,Premier League,Matchweek 8,Sun,Home,L,1.0,2.0,Manchester Utd,...,6,0,1.333333,1.666667,14.333333,3.000000,16.400000,0.666667,0.000000,0.000000
13,2025-10-25,20:00,Premier League,Matchweek 9,Sat,Away,L,2.0,3.0,Brentford,...,5,0,1.000000,2.000000,17.000000,4.000000,15.266667,0.333333,0.000000,0.000000
15,2025-11-01,20:00,Premier League,Matchweek 10,Sat,Home,W,2.0,0.0,Aston Villa,...,5,1,1.333333,2.333333,16.333333,4.333333,15.233333,0.000000,0.000000,0.000000
17,2025-11-09,16:30,Premier League,Matchweek 11,Sun,Away,L,0.0,3.0,Manchester City,...,6,0,1.666667,1.666667,17.666667,5.000000,16.666667,0.333333,0.000000,0.000000


In [76]:
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))

/tmp/ipython-input-578198639.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [77]:
matches_rolling

Date   Time            Comp         Round  \
Team                                                                         
Arsenal                 3  2025-09-13  12:30  Premier League   Matchweek 4   
                        5  2025-09-21  16:30  Premier League   Matchweek 5   
                        7  2025-09-28  16:30  Premier League   Matchweek 6   
                        9  2025-10-04  15:00  Premier League   Matchweek 7   
                        10 2025-10-18  17:30  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 8  2025-10-05  14:00  Premier League   Matchweek 7   
                        9  2025-10-18  15:00  Premier League   Matchweek 8   
                        10 2025-10-26  14:00  Premier League   Matchweek 9   
                        12 2025-11-01  15:00  Premier League  Matchweek 10   
                        13 2025-11-08  20:00  Premier League  Matchweek 11   

                            Day Venue Result   GF   GA         Opponent  ...  \
Team                                                                     ...   
Arsenal                 3   Sat  Home      W  3.0  0.0  Nott'ham Forest  ...   
                        5   Sun  Home      D  1.0  1.0  Manchester City  ...   
                        7   Sun  Away      W  2.0  1.0    Newcastle Utd  ...   
                        9   Sat  Home      W  2.0  0.0         West Ham  ...   
                        10  Sat  Away      W  1.0  0.0           Fulham  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 8   Sun  Home      D  1.0  1.0         Brighton  ...   
                        9   Sat  Away      L  0.0  2.0       Sunderland  ...   
                        10  Sun  Home      L  2.0  3.0          Burnley  ...   
                        12  Sat  Away      L  0.0  3.0           Fulham  ...   
                        13  Sat  Away      L  0.0  3.0          Chelsea  ...   

                            day_code  target  GF_rolling  GA_rolling  \
Team                                                                   
Arsenal                 3          5       1    2.000000    0.333333   
                        5          6       0    2.666667    0.333333   
                        7          6       1    1.333333    0.666667   
                        9          5       1    2.000000    0.666667   
                        10         5       1    1.666667    0.666667   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 8          6       0    0.666667    1.666667   
                        9          5       0    1.000000    1.666667   
                        10         6       0    0.666667    1.333333   
                        12         5       0    1.000000    2.000000   
                        13         5       0    0.666667    2.666667   

                           Sh_rolling SoT_rolling Dist_rolling FK_rolling  \
Team                                                                        
Arsenal                 3   12.333333    2.666667    17.100000   0.666667   
                        5   14.666667    3.333333    15.400000   0.333333   
                        7   13.000000    3.000000    15.033333   0.333333   
                        9   16.000000    5.000000    13.933333   0.000000   
                        10  17.333333    4.666667    14.333333   0.000000   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 8   11.000000    4.000000    14.933333   0.333333   
                        9   10.333333    4.000000    15.933333   0.333333   
                        10  10.333333    3.000000    17.566667   0.666667   
                        12  12.000000    4.000000    18.400000   1.000000   
                        13  11.666667    3.666667    18.300000   1.000000   

                           PK_rolling 

In [78]:
matches_rolling = matches_rolling.droplevel('Team')

In [79]:
matches_rolling.index = range(matches_rolling.shape[0])

In [81]:
# Retraining our model
def predict_matches(matches, predictors):
    train = matches[matches["Date"] < '2025-11-01']
    test = matches[matches["Date"] >= '2025-11-01']
    rf.fit(train[predictors], train["target"])
    predictions = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=predictions), index=test.index)
    precision_score = combined.groupby("predicted").apply(lambda x: (x["actual"] == x["predicted"]).mean())
    return combined, precision_score

In [82]:
combined, precision_score = predict_matches(matches_rolling, predictors + new_cols)

/tmp/ipython-input-1726668432.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  precision_score = combined.groupby("predicted").apply(lambda x: (x["actual"] == x["predicted"]).mean())


In [83]:
precision_score

,0
predicted,
0,0.620690
1,0.363636


In [86]:
combined = combined.merge(matches_rolling[["Date", "Team", "Opponent", "Result"]], left_index=True, right_index=True)

In [87]:
combined

,actual,predicted,Date,Team,Opponent,Result
6,1,0,2025-11-01,Arsenal,Burnley,W
7,0,1,2025-11-08,Arsenal,Sunderland,D
14,0,1,2025-11-01,Aston Villa,Liverpool,L
15,1,0,2025-11-09,Aston Villa,Bournemouth,W
22,0,1,2025-11-02,Bournemouth,Manchester City,L
23,0,0,2025-11-09,Bournemouth,Aston Villa,L
30,0,0,2025-11-01,Brentford,Crystal Palace,L
31,1,0,2025-11-09,Brentford,Newcastle Utd,W
38,1,1,2025-11-01,Brighton and Hove Albion,Leeds United,W
39,0,0,2025-11-09,Brighton and Hove Albion,Crystal Palace,D


In [88]:
class MissingDict(dict):
  __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton",
              "Manchester United": "Manchester Utd",
              "Newcastle United": "Newcastle Utd",
              "Tottenham Hotspur": "Tottenham",
              "West Ham United": "West Ham",
              "Wolverhampton Wanderers": "Wolves"}
mapping = MissingDict(**map_values)

In [90]:
combined["new_team"] = combined["Team"].map(mapping)

In [91]:
from pandas.core.reshape import merge
merged = combined.merge(combined, left_on=["Date", "new_team"], right_on=["Date", "Opponent"])

In [93]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

,count
actual_x,
0,6
1,3


In [94]:
6 / 11

0.5454545454545454